## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
from keras.datasets import cifar10
from tutorial.resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical

In [2]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
# 建立 ResNet 模型
model = resnet(input_shape=(32,32,3)) 
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 16)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [4]:
batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/30
782/782 [==============================] - 29s 38ms/step - accuracy: 0.5139 - loss: 1.8330 - val_loss: 1.7850 - val_accuracy: 0.4956
Epoch 2/30
782/782 [==============================] - 28s 36ms/step - accuracy: 0.6629 - loss: 1.3239 - val_loss: 1.4825 - val_accuracy: 0.6066
Epoch 3/30
782/782 [==============================] - 28s 36ms/step - accuracy: 0.7269 - loss: 1.1077 - val_loss: 1.3775 - val_accuracy: 0.6338
Epoch 4/30
782/782 [==============================] - 28s 36ms/step - accuracy: 0.7713 - loss: 0.9739 - val_loss: 1.5360 - val_accuracy: 0.6134
Epoch 5/30
782/782 [==============================] - 28s 36ms/step - accuracy: 0.8013 - loss: 0.8797 - val_loss: 1.2803 - val_accuracy: 0.6925
Epoch 6/30
782/782 [==============================] - 28s 36ms/step - accuracy: 0.8227 - loss: 0.8077 - val_loss: 1.1426 - val_accuracy: 0.7316
Epoch 7/30
782/782 [==============================] - 28s 36ms/step - accuracy: 0.8431 - loss: 0.7533 - val_loss: 1.5048 - val_accuracy:

In [6]:
## 修改版
model2 = resnet(input_shape=(32,32,3)) 
model2.summary()

model2.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_62 (Conv2D)              (None, 32, 32, 16)   448         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_56 (BatchNo (None, 32, 32, 16)   64          conv2d_62[0][0]                  
__________________________________________________________________________________________________
activation_56 (Activation)      (None, 32, 32, 16)   0           batch_normalization_56[0][0]     
____________________________________________________________________________________________

In [15]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='constant')

train_batches = train_datagen.flow(x_train, y_train,
                                   shuffle=True,
                                   batch_size=batch_size)

valid_datagen = ImageDataGenerator()
valid_batches = valid_datagen.flow(x_test, y_test,
                                   shuffle=True,
                                   batch_size=batch_size)


In [16]:
history = model2.fit(train_batches,
                    steps_per_epoch = len(x_train) / batch_size,
                    validation_data = valid_batches,
                    validation_steps = len(x_test) / batch_size,
                    epochs=epochs,
                    verbose=1)

Epoch 1/30
782/781 [==============================] - 29s 37ms/step - accuracy: 0.2905 - loss: 2.4804 - val_loss: 2.3513 - val_accuracy: 0.3224
Epoch 2/30
782/781 [==============================] - 28s 36ms/step - accuracy: 0.3273 - loss: 2.3646 - val_loss: 2.2645 - val_accuracy: 0.3619
Epoch 3/30
782/781 [==============================] - 28s 36ms/step - accuracy: 0.3555 - loss: 2.2999 - val_loss: 2.2485 - val_accuracy: 0.3793
Epoch 4/30
782/781 [==============================] - 29s 37ms/step - accuracy: 0.3699 - loss: 2.2503 - val_loss: 2.2428 - val_accuracy: 0.3729
Epoch 5/30
782/781 [==============================] - 28s 36ms/step - accuracy: 0.3880 - loss: 2.2085 - val_loss: 2.1836 - val_accuracy: 0.3996
Epoch 6/30
782/781 [==============================] - 28s 36ms/step - accuracy: 0.4012 - loss: 2.1743 - val_loss: 2.1913 - val_accuracy: 0.3969
Epoch 7/30
782/781 [==============================] - 28s 36ms/step - accuracy: 0.4177 - loss: 2.1427 - val_loss: 2.1866 - val_accuracy:

In [17]:
score = model2.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 2.3105366230010986
Test accuracy: 0.49230000376701355
